In [1]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
# Load the Excel file into a Pandas dataframe
road = pd.read_csv('../data/raw/_roads3.csv')
bridge=pd.read_excel('../data/raw/BMMS_overview.xlsx')

In [ ]:
N1Road=road.loc[road.road=='N1']
N1Bridge=bridge.loc[bridge.road=='N1']

In [ ]:
N1Road.head()

In [ ]:
N1Bridge.tail()

In [ ]:
N1Bridge = N1Bridge[["road", "LRPName", "condition", "length", "chainage", "lat", "lon", 'name', 'km','constructionYear']]

In [ ]:
highList=pd.unique(N1Road.road)

In [ ]:
lrpRoad=pd.unique(N1Road.lrp)
lrpBridge=pd.unique(N1Bridge.LRPName)
count=0
missing=[]
for i in lrpBridge:
	if i in lrpRoad:
		count+=1
	else:
		missing.append(i)
print('Number of Coinciding bridges and Road LRPs is : '+str(count))
print('Number of Bridges is : '+str(len(lrpBridge)))
print('Number of Roads is : '+str(len(lrpRoad)))
print('Number of bridges without road LRPs : '+str(len(lrpBridge)-count))
#missing

In [ ]:
N1Bridge['Duplicated'] =N1Bridge.duplicated(subset=['LRPName'])
N1Road['Duplicated']=N1Road.duplicated(subset=['lrp'])

In [ ]:
doubles=[]
for i in N1Bridge.index:
    if N1Bridge.loc[i,'Duplicated'] ==True:
        doubles.append(N1Bridge.loc[i,'LRPName'])
#doubles

In [ ]:
given_value = 'LRP021c'

# find the index of the given value
idx = N1Bridge[N1Bridge['LRPName'] == given_value].index[0]

# print the two values above and below the given value
print(N1Bridge.iloc[max(idx-2, 0):idx+3])

In [ ]:
venn=[]
for i in missing:
    if i in doubles:
        venn.append(i)

In [ ]:
len(venn)

In [ ]:
N1Road["chainage"] = N1Road["chainage"].astype(float)
N1Road["length"] = N1Road["chainage"]*1000 # 20.11680

In [ ]:
for i in range((len(N1Road["length"])-1),0,-1):
    N1Road.loc[i,"length"] = N1Road.loc[i,"length"]-N1Road.loc[i-1,"length"]

In [ ]:
BridgeN1 = N1Bridge.loc[(N1Bridge["chainage"] < 241)]
BridgeN1 = BridgeN1.reset_index(drop=True)

In [ ]:
temp=[]
for i in range(1,len(BridgeN1["road"])):
    if len(BridgeN1["name"][i]) > 4:
        #temp.append(BridgesN1['name'])     #to see the names of the bridges and identify right and left side and how they occur
        if BridgeN1["name"][i][-2:] == 'L)' or BridgeN1["name"][i][-4:] == 'eft)' or BridgeN1["name"][i][-3:] == 'L )' or BridgeN1["name"][i][-4:] == 'EFT)':
            #print(BridgesN1['name'][i][-3:])        #tinker with the last bracket to see the end of the names
            BridgeN1 = BridgeN1.drop(i,axis = 0)
        
         

In [ ]:
BridgeN1.shape

In [ ]:
BridgeN1 = BridgeN1.sort_values(by=['road','km','constructionYear'], ascending=False) 
BridgeN1 = BridgeN1.drop_duplicates(subset=['road', 'km'], keep='first')

In [ ]:
BridgeN1.shape

In [ ]:
BridgeN1 = BridgeN1.rename(columns={"LRPName": "lrp"})

In [ ]:
N1 = pd.merge(N1Road, BridgeN1, how="outer", on=["road", "lrp", "length", "lat", "lon", "chainage"])

In [ ]:
N1.head()

In [ ]:
# add type of road/bridge
N1["model_type"] = N1["lrp"].apply(lambda x: "sink" if x == "LRPS" else ("source" if x == "LRPE" else "link"))
N1.loc[N1["condition"].notnull(), "model_type"] = "bridge"

In [ ]:
N1 = N1.sort_values(by=['road','chainage'],ascending = False)

In [ ]:
N1 = N1[2:]
N1 = N1.reset_index(drop=True)

In [ ]:
N1.loc[0,"model_type"] = "source"

In [ ]:
N1["bridge_name"] = N1["name_y"]
N1["name_y"] = N1["model_type"]

In [ ]:
N1["id"] = range(len(N1))

In [ ]:
N1.head()

In [ ]:
N1 = N1[["road", "id", "model_type", "name_y", "lat", "lon", "length", "condition", "bridge_name"]]

In [ ]:
N1 = N1.rename(columns={"name_y": "name"})

In [ ]:
N1[N1["road"] == "N1"].to_csv("../data/processed/N1.csv", index=None)

In [ ]:
N1Road=road.loc[road.road=='N1']

In [ ]:
N1Road["chainage"] = N1Road["chainage"].astype(float)
N1Road["length"] = N1Road["chainage"]*1000 # 20.11680

In [ ]:
for i in range((len(N1Road["length"])-1),0,-1):
    N1Road.loc[i,"length"] = N1Road.loc[i,"length"]-N1Road.loc[i-1,"length"]

In [ ]:
N1Road.head()

In [ ]:
N1Road['id'] = N1Road.index
N1Road['model_type'] = 'link'
N1Road.loc[N1Road['gap'] == 'BS', 'model_type'] = 'bridge'


In [ ]:
N1Road.head(16)

In [ ]:
N1Road=N1Road[["road","lrp", "id", "model_type", "name", "lat", "lon", "length"]]

In [ ]:
N1Road=N1Road[["road", "id", "model_type", "name", "lat", "lon", "length"]]

In [ ]:
N1Road.tail()

In [ ]:
N1Road.loc[0,"model_type"] = "sink"
N1Road.loc[1338,"model_type"] = "source"
N1Road.head()

In [ ]:
N1Road = N1Road.sort_values(by=['road','id'],ascending = False)
N1Road = N1Road.reset_index(drop=True)
N1Road['id'] = N1Road.index

In [ ]:
N1Road.head()

In [ ]:
N1Road[N1Road["road"] == "N1"].to_csv("../data/processed/N1trial.csv", index=None)